In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
base_url = "http://localhost:11434"
model = "sheldon:latest"
llm = ChatOllama( model = model)
llm

ChatOllama(model='sheldon:latest')

In [2]:
template = ChatPromptTemplate.from_template("{prompt}")
chain = template | llm | StrOutputParser()

about = "My name is Lexi Kant. I work for a tech company"
response = chain.invoke({'prompt': about})
print(response)

(sitting up straight, looking around the room) Ah, Lexi Kant, nice to meet you, Lexi Kant. A tech company, how...predictable. (pausing to adjust his glasses) I must say, I'm intrigued by your profession. What specific area of technology do you work in? (leaning back in his chair, steepling his fingers together)


In [3]:
prompt = "What is my name"
chain.invoke({"prompt": prompt})

"Don't bother me with your trivial questions, but if you must know, it's Leonard Hofstadter. Now, if you'll excuse me, I have more important things to attend to, such as solving this intriguing math problem or practicing my violin. Bazinga!"

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [5]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db")

In [6]:
runnable_with_history = RunnableWithMessageHistory(chain,get_session_history)

In [7]:
user_id = 'kenny luu'
history = get_session_history(user_id)
history.get_messages()

C:\Users\khain\AppData\Local\Temp\ipykernel_21408\2444107318.py:2: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  history = get_session_history(user_id)


[]

In [8]:
history.clear()

In [9]:
runnable_with_history.invoke([HumanMessage(content=about)], 
                             config = {'configurable':{'session_id':user_id}})

"Lexi Kant. (pausing to adjust his glasses) Ah, a tech company, you say? (looking at the message) Fascinating. What specific area of technology do you work in? And what makes your company notable? I'm sure it's not just another mindless corporation like Caltech, where I'm forced to work with... (pausing to emphasize the word) ...idiots."

In [10]:
runnable_with_history.invoke([HumanMessage(content="What's my name")], 
                             config = {'configurable':{'session_id':user_id}})

'[HumanMessage(content="Lexi Kant. (smiling politely) Ah, nice to meet you, Lexi. My name is Sheldon Cooper, by the way. I\'m a theoretical physicist and the head of research at Caltech.", additional_kwargs={}, response_metadata={})]'

## Better formatting

In [11]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.messages import HumanMessage, SystemMessage

In [12]:
system = SystemMessagePromptTemplate.from_template("You are helpful assistant.")
human = HumanMessagePromptTemplate.from_template("{input}")

messages = [system, MessagesPlaceholder(variable_name = 'history'), human]
prompt = ChatPromptTemplate(messages)
chain = prompt | llm | StrOutputParser()

runnable_with_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key = 'input', history_messages_key = 'history')

In [13]:
def chat_with_llm(session_id, input): 
    output = runnable_with_history.invoke(
        {'input': input},
         config = {'configurable':{'session_id':user_id}}
    )
    return output

In [14]:
user_id = 'test'

In [15]:
chat_with_llm(user_id, "what is my name")

"I don't have any information about you, and I'm not able to access any personal data about you. This conversation just started, and I'm here to help with any questions or topics you'd like to discuss. Is there something specific you'd like to talk about, or do you want to just start fresh?"